In [1]:
# Import required libraries
import google.generativeai as genai
import textwrap
from IPython.display import Markdown, display
import os
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

# Configure Gemini Pro
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-1.5-pro')

/home/ofge/advanced-prompt/adv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define our prompt templates
PROMPT_TEMPLATES = {
    "document_analysis": """
    You are a legal document analysis assistant. Your task is to analyze the provided legal document 
    using careful, step-by-step reasoning. Follow these steps:

    1. Identify the type of document (e.g., NDA, Employment Agreement, Lease)
    2. List all major sections with brief descriptions
    3. For each key clause:
       a. Explain its purpose in plain language
       b. Note any unusual or concerning language
       c. Suggest potential implications
    4. Highlight any red flags or areas requiring legal review

    Document: {document_text}

    Structure your response as follows:
    - Document Type: [type]
    - Sections:
      - [section name]: [description]
    - Key Clauses:
      - [clause name]:
        * Purpose: [explanation]
        * Concerns: [if any]
        * Implications: [analysis]
    - Red Flags: [list of potential issues]
    - Summary: [overall assessment]
    """,
    
    "term_explanation": """
    Explain the following legal term as if to a non-lawyer, using simple analogies and examples.
    Provide 1-2 sentences of definition and 1-2 sentences of practical implications.

    Term: {term}

    Examples of good explanations:
    - "Indemnification": This means one party agrees to cover the costs if the other party gets sued. 
      It's like agreeing to pay your friend's parking tickets if you borrowed their car.
    - "Force Majeure": This clause excuses parties when unforeseen events (like natural disasters) prevent 
      fulfilling the contract. Similar to not being penalized for missing work during a hurricane.
    
    Now explain the requested term in the same style:
    """
}

In [3]:
# Helper functions
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def analyze_legal_document(document_text):
    """Main function to analyze legal documents using chain-of-thought prompting"""
    prompt = PROMPT_TEMPLATES["document_analysis"].format(document_text=document_text)
    response = model.generate_content(prompt)
    return response.text

def explain_legal_term(term):
    """Explain legal terms using few-shot learning"""
    prompt = PROMPT_TEMPLATES["term_explanation"].format(term=term)
    response = model.generate_content(prompt)
    return response.text

In [10]:
# Sample NDA document
sample_nda = """
NON-DISCLOSURE AGREEMENT

This Non-Disclosure Agreement (the "Agreement") is made and entered into as of the date of last signature below by and between:
1. Disclosing Party: ABC Corporation ("Discloser")
2. Receiving Party: XYZ Inc. ("Recipient")

1. Definition of Confidential Information
For purposes of this Agreement, "Confidential Information" means any data or information that is proprietary to the Discloser and not generally known to the public, whether in tangible or intangible form.

2. Obligations of Recipient
Recipient agrees to maintain the Confidential Information in strictest confidence and not to disclose such information to any third parties. This obligation shall continue for a period of 5 years from the date of disclosure.

3. Exclusions
The obligations of confidentiality shall not apply to information that:
a) was known to Recipient prior to disclosure
b) becomes publicly available through no fault of Recipient
c) is independently developed by Recipient

4. Remedies
The parties acknowledge that unauthorized disclosure of Confidential Information may cause irreparable harm for which monetary damages would be inadequate. Discloser shall be entitled to seek injunctive relief in addition to other legal remedies.

5. Governing Law
This Agreement shall be governed by the laws of the State of California without regard to its conflict of laws provisions.

IN WITNESS WHEREOF, the parties have executed this Agreement as of the Effective Date.
"""



In [11]:
# Analyze the document
analysis_result = analyze_legal_document(sample_nda)
display(to_markdown(analysis_result))

> - Document Type: Non-Disclosure Agreement (NDA)
> 
> - Sections:
>     - Definition of Confidential Information: Defines what is considered confidential.
>     - Obligations of Recipient: Outlines the recipient's responsibilities regarding the confidential information.
>     - Exclusions: Specifies situations where the confidentiality obligations do not apply.
>     - Remedies: Details the legal recourse available to the Discloser in case of a breach.
>     - Governing Law:  Specifies the jurisdiction governing the agreement.
> 
> - Key Clauses:
>     - Definition of Confidential Information:
>         * Purpose: To clearly define the scope of information protected by the NDA.
>         * Concerns: The definition is quite broad ("any data or information...proprietary to the Discloser").  It lacks specific examples and could be interpreted very widely.
>         * Implications: The broad definition could encompass information that is not truly confidential, potentially creating unnecessary restrictions on the Recipient.
> 
>     - Obligations of Recipient:
>         * Purpose: To establish the Recipient's duty to protect the Confidential Information.
>         * Concerns:  While "strictest confidence" is a common phrase, it lacks specific actions. How is "strictest confidence" measured?
>         * Implications:  The vague wording might make it difficult to prove a breach.  The 5-year term is relatively standard but could be shorter or longer depending on the nature of the information.
> 
>     - Exclusions:
>         * Purpose: To carve out exceptions to the confidentiality obligations, preventing the NDA from being overly restrictive.
>         * Concerns: The "independently developed" exclusion could be a source of dispute.  How is independent development proven?
>         * Implications: This clause is essential to protect the Recipient's pre-existing knowledge and independent creations, but its effectiveness depends on clear documentation and potential witness testimony.
> 
>     - Remedies:
>         * Purpose: To specify the consequences of breaching the NDA and deter unauthorized disclosure.
>         * Concerns:  Standard language, but the threat of injunctive relief can be powerful and should be taken seriously by the Recipient.
>         * Implications: This clause strengthens the Discloser's position and gives them significant leverage in case of a breach.
> 
>     - Governing Law:
>         * Purpose: To determine which state's laws will interpret and enforce the agreement.
>         * Concerns: None. California is a common choice for governing law, especially for businesses located there.
>         * Implications:  This choice of law will determine the legal framework applied in any disputes related to the NDA.
> 
> 
> - Red Flags:
>     - Overly broad definition of "Confidential Information."
>     - Vague description of the Recipient's obligations ("strictest confidence").
>     - Potential for disputes regarding "independently developed" information.
> 
> - Summary:
> This NDA is a relatively standard agreement, but the broad definition of "Confidential Information" and the vague wording regarding the Recipient's obligations create potential ambiguity. It would be advisable for both parties to clarify these points before signing.  The Recipient should consider requesting more specific examples of what constitutes Confidential Information and clearer guidance on the practical steps required to maintain "strictest confidence."  The Discloser should consider refining the definition to avoid overreach and potential future disagreements.  Legal review is recommended for both parties, especially the Recipient, before signing.


In [ ]:
# Test term explanation
term = "Injunctive relief"
explanation = explain_legal_term(term)
display(to_markdown(explanation))

> "Injunctive Relief": This is a court order that requires someone to do something or stop doing something.  Think of it like a judge telling a noisy neighbor to stop playing loud music at 3 a.m., or ordering a company to stop polluting a river.  The practical implication is that failing to comply with an injunction can lead to serious consequences, such as fines or even jail time.


In [6]:
# Edge case handling examples
def handle_edge_cases(document_text):
    """Handle various edge cases in document analysis"""
    
    # Case 1: Empty document
    if not document_text.strip():
        return "Error: Empty document provided"
    
    # Case 2: Non-legal document
    prompt = f"""Determine if the following text appears to be a legal document. 
    Respond only with 'yes' or 'no':\n\n{document_text}"""
    response = model.generate_content(prompt).text.lower()
    
    if 'no' in response:
        return "Warning: This doesn't appear to be a legal document. Analysis may be unreliable."
    
    # Case 3: Very long document
    if len(document_text.split()) > 2000:
        return "Note: Document is very long. Consider analyzing sections separately for best results."
    
    # Case 4: Poor quality scan (many OCR errors)
    error_threshold = 0.1  # 10% of words contain numbers/special chars
    words = document_text.split()
    unusual_words = [w for w in words if any(c.isdigit() or c in '@#$%^&*' for c in w)]
    
    if len(unusual_words)/len(words) > error_threshold:
        return "Warning: Document may contain scanning errors. Results may be inaccurate."
    
    return None

# Test edge cases
edge_case_result = handle_edge_cases("This is just some random text, not a legal document")
print(edge_case_result)

In [8]:
# Test cases
test_cases = [
    {
        "name": "Complete NDA",
        "document": sample_nda,
        "expected": ["Non-Disclosure", "Confidential Information", "5 years"]
    },
    {
        "name": "Empty Document",
        "document": "",
        "expected": "Error"
    },
    {
        "name": "Non-Legal Text",
        "document": "The quick brown fox jumps over the lazy dog.",
        "expected": "Warning"
    }
]

In [9]:


# Evaluation function
def evaluate_system(test_cases):
    results = []
    for case in test_cases:
        edge_case_check = handle_edge_cases(case["document"])
        
        if edge_case_check and ("Error" in edge_case_check or "Warning" in edge_case_check):
            passed = edge_case_check.startswith(case["expected"])
            results.append({
                "test": case["name"],
                "passed": passed,
                "output": edge_case_check
            })
            continue
            
        analysis = analyze_legal_document(case["document"])
        passed = all(keyword in analysis for keyword in case["expected"])
        results.append({
            "test": case["name"],
            "passed": passed,
            "output": analysis[:200] + "..." if len(analysis) > 200 else analysis
        })
    
    return results

# Run evaluation
evaluation_results = evaluate_system(test_cases)
for result in evaluation_results:
    print(f"Test: {result['test']}")
    print(f"Passed: {result['passed']}")
    print(f"Output: {result['output']}\n")

Test: Complete NDA
Passed: False
Output: - **Document Type:** Non-Disclosure Agreement (NDA)

- **Sections:**
    - **Definition of Confidential Information:** Defines what information is protected by the agreement.
    - **Obligations of Re...

Test: Empty Document
Passed: True
Output: Error: Empty document provided

Test: Non-Legal Text
Passed: True
Output: Warning: This doesn't appear to be a legal document. Analysis may be unreliable.

